<a href="https://colab.research.google.com/github/systemis/challenge-2/blob/main/code2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import pandas as pd 
import matplotlib.pyplot as plt
from os import listdir 
from os.path  import isfile, join
from google.colab import files

In [2]:
data_name_list = [f for f in listdir('/content/drive/MyDrive/data') if isfile(join('/content/drive/MyDrive/data', f))]


In [5]:
height = 600 
width = 463

# 463, 600

character_table = ['A', 'B', 'C', 'D', 'E']
answer = 5 
questions_part = 5
total_part = 6
ques_each_side = 30
total_question = 60

statistical_table = [0] * total_question

def generateColumns(): 
  columns = ['Student ID', 'Surname', 'First Name', 'Code']
  for i in range(60): columns.append('Question: '+str((i+1)))
  columns.append('Score')
  return columns

def grid(): 
  _grid = [] 
  for i in range(total_question): 
    _grid.append([])
    for j in range(answer): 
      _grid[i].append(0)
  return _grid

def show_images(images):
    for image in images: 
        imshow = plt.imshow(image)
        plt.show()



In [7]:
img_filenames = data_name_list
img_list = list()
student_data_list = list()
for name in img_filenames: 
  if name == '.DS_Store': continue
  img = plt.imread('/content/drive/MyDrive/data/' + name)
  img = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA)
  img = img[90:, :]
  img_list.append(img)
  
  name = name.replace('.png', '').split('_')
  if len(name) < 4: 
    print(name)
    continue
  student = [name[0], name[1], name[2], name[3]]
  student_data_list.append(student)

def get_rect_list(contours): 
  rect_list = list() 
  for p_c in contours: 
    arc_length = cv2.arcLength(p_c, True)
    approx = cv2.approxPolyDP(p_c, arc_length * 0.02, True)
    if approx == 4: 
      rect_list.append(p_c)
  return rect_list

def check_part(sides, side_pos, pc, result_data): 
  part = sides[side_pos][(80*pc):80*(pc+1), :]
  part = part[5:70, :]
  gray = cv2.cvtColor(part, cv2.COLOR_BGR2GRAY)
  _, threshold = cv2.threshold(gray, 20, 5, cv2.THRESH_BINARY_INV)
  
  row_height = part.shape[0] // questions_part
  col_width = part.shape[1] // answer
  part_gray = cv2.cvtColor(part, cv2.COLOR_BGR2GRAY)
  # show_images([part])
  part = part_gray
  for i in range(questions_part): 
    current_question = (questions_part*pc + i + 1) + ques_each_side*side_pos
    row = part[row_height*i:row_height*(i+1), :]
    for j in range(answer): 
      col = row[:, col_width*j:col_width*(j+1)]
      count = cv2.countNonZero(col)
      if count < 300: 
        result_data[current_question - 1][j] = 1
        


In [8]:
def detect_checked(img, only_firstfive=False): 
  # img = img_list[0]
  img_copy = img.copy()
  left_side = img[:, 80:200]
  right_side = img[:, 290:410]
  gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  blur_img = cv2.GaussianBlur(gray_img, (5, 5), 0)

  left_side = left_side[:490, :]
  right_side = right_side[:490, :]
  sides = [left_side, right_side]

  result_data = grid()
  if only_firstfive: 
    for side_pos in range(1): 
      part_height = sides[side_pos].shape[0] // total_part
      for pc in range(1): 
        check_part(sides, side_pos, pc, result_data)
    return result_data
  for side_pos in range(2): 
    part_height = sides[side_pos].shape[0] // total_part
    for pc in range(total_part): 
      check_part(sides, side_pos, pc, result_data)
  return result_data



In [9]:
def detectAnswerData(): 
  answer_data = ['' for i in range(total_question)]
  answer_image = plt.imread('/content/drive/MyDrive/3A.png')
  answer_image = cv2.resize(answer_image, (width, height), interpolation = cv2.INTER_AREA)
  answer_image = answer_image[90:, :]
  checked_data = detect_checked(answer_image) 
  # show_images([answer_image])
  for i in range(len(checked_data)): 
    for j in range(len(checked_data[i])): 
      if checked_data[i][j]: 
        answer_data[i] = character_table[j]
  return answer_data

answer_data = detectAnswerData()


def grade(checked_data, student, is_statistical=False): 
  score = 0 
  check =0 
  for (question_index, question_data) in enumerate(checked_data): 
    for j in range(len(question_data)): 
      if question_data[j] == 1: 
        result = character_table[j]
        student.append(result)
        check += 1 
        if result == answer_data[question_index]: 
          if is_statistical: statistical_table[question_index] += 1 
          score += 1 
        break
  return score

def checkDataInFirstFive(): 
  student_data_copy = student_data_list.copy()
  for (i, image) in enumerate(img_list): 
    student = student_data_copy[i].copy()
    checked_data = detect_checked(image, only_firstfive=True)
    checked_data = checked_data[:5]
    for question_data in checked_data: 
      for j in range(len(question_data)): 
        if question_data[j] == 1: 
          student.append(character_table[j])
    student_data_copy[i] = student

  columns = ['Student ID', 'Surname', 'First Name', 'Code']
  for i in range(5): columns.append('Question: '+str((i+1)))
  data_frame = pd.DataFrame(student_data_copy, columns=columns)
  print('--- Checked of first five question of all student: -----')
  print(data_frame)
  print('\n')

#Question 5

In [15]:
def gradingStudentInClass(): 
  global statistical_table
  student_data_copy = student_data_list.copy()
  for i in range(len(student_data_copy)): 
    image = img_list[i]
    student = student_data_copy[i].copy()
    checked_data = detect_checked(image)
    score = grade(checked_data, student, True)
    score = (score / total_question) * 100
    student.append(score)
    student_data_copy[i] = student
  
  columns = generateColumns()
  data_frame = pd.DataFrame(student_data_copy, columns=columns)

  grading_columns = ['ID', 'Score']
  grading_data = [[item[0], item[-1]] for item in student_data_copy]
  print('---- Generate grading score of class: Done ----\n')
  # print(student_data_copy)
  for i in range(len(student_data_copy)): 
    if len(student_data_copy[i]) > 65: 
      print(student_data_copy[i])
  grading_data_frame = pd.DataFrame(grading_data, columns=grading_columns)
  grading_data_frame.to_csv('grading.csv')
  return grading_data_frame

In [16]:
score = gradingStudentInClass()

---- Generate grading score of class: Done ----



In [17]:
score

,ID,Score
0,2000129,46.666667
1,2000104,28.333333
2,2000318,25.000000
3,20020027,25.000000
4,2000147,18.333333
...,...,...
76,2000140,23.333333
77,2000112,28.333333
78,2000136,25.000000
79,2000114,18.333333


In [18]:
score.to_csv()

',ID,Score\n0,2000129,46.666666666666664\n1,2000104,28.333333333333332\n2,2000318,25.0\n3,20020027,25.0\n4,2000147,18.333333333333332\n5,2000109,25.0\n6,2000111,25.0\n7,2000137,21.666666666666668\n8,20003,20.0\n9,2000150,30.0\n10,2000105,20.0\n11,2000102,25.0\n12,2000127,51.66666666666667\n13,2000342,21.666666666666668\n14,2000310,18.333333333333332\n15,2000107,18.333333333333332\n16,2000312,46.666666666666664\n17,2000339,26.666666666666668\n18,2000133,43.333333333333336\n19,2000130,48.333333333333336\n20,2000106,25.0\n21,2000101,28.333333333333332\n22,2000118,75.0\n23,20020029,25.0\n24,2000113,23.333333333333332\n25,2000348,26.666666666666668\n26,2000149,26.666666666666668\n27,2000123,65.0\n28,2000119,100.0\n29,2000322,23.333333333333332\n30,2000325,30.0\n31,2000329,25.0\n32,20020030,20.0\n33,2000120,71.66666666666667\n34,2000139,26.666666666666668\n35,2000135,21.666666666666668\n36,2000143,18.333333333333332\n37,20020031,28.333333333333332\n38,20020033,23.333333333333332\n39,2000108,